Universidade Federal de Alagoas

Instituto de Computação


Disciplina: Sistemas Distribuídos

Semestre letivo: 2018.1

Professor: André Lage Freitas


# Trabalho Prático AB2

**Importante**. Leia atentamente as especificações pois só serão aceitos trabalhos que as obedecerem.


## Parte 1: Implementação de um monitor de caraga da trabalho descentralizado

**Implementar uma versão modificada do detector de falha não confiável** (_unreliable failure detector_) descrito na Seção _15.1.1 Failure assumptions and failure detectors_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

A **modificação** consiste em enviar informações sobre a carga do sistema (_system workload_) em vez de informações sobre falhas que indicariam os processos suspeitos e não-suspeitos. Para isso, em vez de cada processo enviar mensagem do tipo `I'm p and I'm alive`, os processos enviarão a medição interna da sua carga de trabalho. 

Para simular a medição da carga de trabalho dos processos, utilize números aleatórios entre `0` e `1` com precisão de uma casa decimal apenas, por exemplo, `0.4`.

Utilize como período de envio das mensagens `2` segundos.

### Detalhes de implementação

Cada processo do sistema deverá ser implementado como um Julia Worker. O Master será apenas o gestor do experimento, ou seja, será responsável para criar os processos (Julia Workers) e iniciar o sistema. Uma vez iniciado, o sistema deve funcionar por si sem depender do Master; uma vez que o algoritmo é descentralizado.

Para fins de visualização do monitoramento da carga de trabalho, os **Workers devem enviar ao Master** periodicamente (a cada `2` segundos) uma mensagem com o estado dos workers que eles monitoram (incluindo ele mesmo). Essa mensagem deve ser impresso na tela do master da seguinte forma.

```julia
[...]
Workload from Worker 4: [0.4,0.9,0.7]
Workload from Worker 2: [0.4,0.9,0.7]
Workload from Worker 3: [0.4,0.9,0.7]
[...]
```

A última linha acima significa que as cargas de trabalho dos Workers monitorados pelo Worker 3 são `0.4,0.9,0.7`, ou seja, os Workers têm as seguintes carga de trabalho:

* Worker 2 = 0.4
* Worker 3 = 0.9
* Worker 4 = 0.7

Lembre-se de que, em Julia, o ID do Master é 1 e os Workers têm ID maiores que 1.

## Parte 2: Implementação de um algoritmo de Eleição

**Implementar o algoritmo de Eleição** baseado em anel (_ring-based election algorithm_) descrito na Seção _15.3 Elections_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

Considere as seguintes condições.

1. Uma eleição nova é disparada **quando a carga de trabalho do líder for maior ou igual a `0.8`** e **o Worker que identificou essa alta carga de trabalho do líder tenha carga de trabalho menor que `0.8`**.
1. Em caso de empate, o Worker com ID mais alto ganha a eleição.
1. Caso não haja Worker com carga de trabalho menor que `0.8` para liderar, escolhe-se o Worker com menor carga de trabalho.

Quando um líder for eleito, ele deve enviar a a seguinte mensagem ao Master:

```julia
[...]
New leader elected! Worker 2
[...]
```


### Suporte de programação distribuída em Julia

Para utilizar o suporte de programação distribuída em Julia, primeiro devemos carergar a biblioteca `Distributed`:

```julia
using Pkg
Pkg.add("Distributed")
using Distributed
```

A seguir, segue uma lista (não extensiva) de funções que poderão ajudar na programação distribuída para a implementação do trabalho:

```julia
addprocs
remotecall
@fetchfrom
@everywhere
sleep
@spawn
```

Para buscar uma rápida ajuda sobre as funções, utilize `?` na frente da função (ver exemplo abaixo). 

```julia
?addprocs
```

Utilize o [material do mini-curso do Professor](https://github.com/proflage/2018-julia-hands-on) para estudo e a [documentação oficial da linguagem Julia (1.0)](https://docs.julialang.org/en/stable/) para demais dúvidas.

## Código-fonte

Documente seu código em comentários no próprio código-fonte.

**Todo o seu código tem que estar colado na próxima célula**, mesmo se ele tenha sido feito em IDEs, células separadas, etc. O Professor testará a apenas o que estiver colado na próxima célula.

In [1]:
# Espaço para escrever o código fonte.


using Pkg
Pkg.add("Distributed")
using Distributed
addprocs(7)


# Global variables
@everywhere workloads = fill(0.0, nworkers()+1)
@everywhere leader = nworkers()+1
@everywhere candidate = -1


#=
    Description: This function prints the requested messages on the master screen
    Parameters: It has a single parameter of type "String" that represents the message
    Return: The message on the master screen
=#
function sendMaster(message)
    println(message)
end



#=
    Description: This function updates the global variable "leader" to receive the new elected leader "new_leader"
    Parameters: "new_leader" representing the new elected leader
    Return: Assignment of the new leader to the global variable "leader"
=#
@everywhere function updateLeader(new_leader)
    leader = new_leader
end


#=
    Description: This function prints on the master screen the message of the new elected leader and
                 spreads to all workers the new elected leader
    Parameters: "future_leader" representing the next leader
    Return: Print on the master screen of the specified message and update the new leader on all workers

=#
@everywhere function broadcastNewLeaderElected(future_leader)
    message = string("New leader elected! Worker ", future_leader)
    saveInFile(message)
    @fetchfrom 1 sendMaster(message)
    for pid in workers()
        remotecall(updateLeader, pid, future_leader)
    end
end

#=
    Description: This function is responsible for performing the election algorithm,
                going through all the workers respecting the order of the ring and checking their workloads
    Parameters:
        "id_candidate"  which is the id of a candidate for new leader
        "cand_workload" which is their respective workload
=#
@everywhere function ringElection(id_candidate, cand_workload)
    current_id = myid()

    if (id_candidate == current_id)

        if(leader != current_id)
            data = string("Next Leader: Worker ", id_candidate)
            saveInFile(data)
            broadcastNewLeaderElected(current_id)
        end

    elseif (workloads[current_id] < cand_workload)
        next_worker = findNextWorker(current_id)
        candidate = current_id
        remotecall(ringElection, next_worker, candidate, workloads[current_id])

    elseif (workloads[current_id] == cand_workload)
        next_worker = findNextWorker(current_id)

        if(id_candidate > current_id)
            candidate = id_candidate
            remotecall(ringElection, next_worker, candidate, cand_workload)
        else
            candidate = current_id
            remotecall(ringElection, next_worker, candidate, workloads[current_id])
        end

    else
        next_worker = findNextWorker(current_id)
        candidate = id_candidate
        remotecall(ringElection, next_worker, candidate, cand_workload)
    end

end


#=
    Description: This function only calculates the next worker of the logical ring,
                 if the current worker is the last, returns to the beginning of the ring
    Parameters: "current_worker" that is the current worker where the election is going
    Return: The next worker of the ring
=#
@everywhere function findNextWorker(current_worker)
    next_worker = current_worker + 1
    if(next_worker > workers()[end])
        next_worker = 2
    end
    return next_worker
end


#=
    Description: This function is responsible for initiating the election from the worker who identified the high load of the leader
    Parameters:
        "current_worker": current worker who identified the high workload of the leader
        "current_workload": current workload of this worker who identified the high workload of the leader
    Return: The current worker is assigned the candidate global variable,
            the next worker is calculated and a remote call is made to the next worker
=#
@everywhere function startRingElection(current_worker, current_workload)
    candidate = current_worker
    next_worker = findNextWorker(current_worker)
    remotecall(ringElection, next_worker, candidate, current_workload)
end


#=
    Description: This function is responsible for monitoring the workload, printing on the master
                screen the workloads of the workers and checking the conditions of election
    Parameters:
        "worker_id" that represents the id of a worker in the list of workers
        "worker_workload" is their respective workload at that moment
    Return: prints the specified messages on the master screen and calls function startRingElection if conditions are met

=#
@everywhere function checkElectionConditions(worker_id, worker_workload)
    current_id = myid()
    workloads[worker_id] = worker_workload

    if (current_id == 1)
        message = string("Workload from Worker ", worker_id, ": ", workloads[2:nworkers()+1])
        saveInFile(message)
        @fetchfrom current_id sendMaster(message)


    elseif (worker_id == leader && worker_workload >= 0.8 && workloads[current_id] < 0.8)
            saveInFile(string("Ring election started! The current leader Worker ", worker_id, " has workload [", worker_workload, "] and I, Worker ", current_id, ", am a candidate with workload [", workloads[current_id],"]"))
            startRingElection(current_id, workloads[current_id])
    end
end

#=
    Description: This function starts the monitoring of worker loads, and sends workers' workload
                 information to all processes (workers and master) every 2 seconds
    Parameters: None
    Return: A call to function checkElectionConditions()
=#
@everywhere function startWorkloadMonitor()
    current_id = myid()

    while true 
        sleep(2)
        current_workload = round(rand(), digits = 1)
        for pid in procs()
            remotecall(checkElectionConditions, pid, current_id, current_workload)
        end
    end
end

# This funcion creates an "log.txt" file in the current directory and write the data
@everywhere function saveInFile(data)
    open("log.txt", "a") do file
        write(file, data, "\n\n")
    end
end


saveInFile(string("------------------------------------------------------------------------------"))
saveInFile("Output File of TP AB2 - Distributed Systems (IC/UFAL)")
saveInFile(string("Initial Leader: ", leader))

# Start system
message = string(" ===> Initial Leader: Worker ", leader)
@fetchfrom 1 sendMaster(message)

message = string(" ===> The log.txt file was created and is being updated.")
@fetchfrom 1 sendMaster(message)

for pid in workers()
    remotecall(startWorkloadMonitor, pid)
end


  Updating registry at `/home/jrun/.julia/registries/JuliaPro`
  Updating git-repo `https://pkg.juliacomputing.com/registry/JuliaPro`
 Resolving package versions...
  Updating `~/.julia/Project.toml`
 [no changes]
  Updating `~/.julia/Manifest.toml`
 [no changes]
 ===> Initial Leader: Worker 8
 ===> The log.txt file was created and is being updated.
Workload from Worker 5: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Workload from Worker 2: [0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Workload from Worker 4: [0.1, 0.0, 0.6, 0.0, 0.0, 0.0, 0.0]
Workload from Worker 6: [0.1, 0.0, 0.6, 0.0, 0.3, 0.0, 0.0]
Workload from Worker 7: [0.1, 0.0, 0.6, 0.0, 0.3, 0.1, 0.0]
Workload from Worker 8: [0.1, 0.0, 0.6, 0.0, 0.3, 0.1, 0.7]
Workload from Worker 3: [0.1, 0.3, 0.6, 0.0, 0.3, 0.1, 0.7]
Workload from Worker 8: [0.1, 0.3, 0.6, 0.0, 0.3, 0.1, 0.2]
Workload from Worker 2: [0.5, 0.3, 0.6, 0.0, 0.3, 0.1, 0.2]
Workload from Worker 5: [0.5, 0.3, 0.6, 0.7, 0.3, 0.1, 0.2]
Workload from Worker 4: [0.5, 0.3, 0.6, 0.7, 0.3


## Entrega

As respostas deverão ser entregues na parte indicada desse arquivo reservada ao código fonte, no formato [IJulia Notebook](https://github.com/JuliaLang/IJulia.jl), que utiliza tecnologia [Jupyter](https://www.jupyter.org).

O programa deve ser implementado na linguagem de programação [**Julia** versão 1.0](https://docs.julialang.org/en/stable/). Seu trabalho será testado na [JuliaBox](https://juliabox.com). 

Baixe seu arquivo `.ipynb` e anexe-o ao Google Classroom. Não se esqueça de **testar seu arquivo `.ipynb` IJulia na JuliaBox** antes de enviá-lo.


### Forma 

O arquivo IJulia Notebook deverá ser entregue ao Professor **exclusivamente através do Google Classroom**.



A responsabilidade sobre a integridade do arquivo contendo trabalho é exclusivamente dos discentes. Serão ignorados os trabalhos cujos arquivos não conseguirem ser abertos pelo Professor.

### Prazos

O prazo de entrega está descrito no **Google Classroom**.


## Pontuação extra

O(a) discente que realizar mais tarefas, além do que foi especificado neste trabalho, o professor atribuirá de 0,5 a 1,0 ponto extra a depender da relevância da contribuição no programa. O critério será decidido pelo professor. 

O(a) discente deve indicar qual é a tarefa executada. Por exemplo, adição de funcionalidades, armazenamento de dados em arquivo, documentação de código, comentários em inglês sem erros ortográficos, etc.


**Plágio** A nota zero será atribuída caso haja qualquer tipo de cópia parcial ou integral assim como as devidas medidas legais. Leia a [cartilha sobre plágio](http://www.noticias.uff.br/arquivos/cartilha-sobre-plagio-academico.pdf).

## Disclaimer

Esse material foi elaborado pelo [Prof. André Lage Freitas](https://sites.google.com/a/ic.ufal.br/andrelage/) e está licenciado sob a licença _GNU General Public License v3.0_.